In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import time
import requests
import bs4
import json

# read and merge 3 dfs

In [47]:
df1 = pd.read_csv('data/EUW/euchall.csv',low_memory=False)
print(df1.shape)

(11127, 1552)


In [48]:
df2 = pd.read_csv('data/EUW/eugm.csv',low_memory=False)
print(df2.shape)

(3000, 1552)


In [49]:
df3 = pd.read_csv('data/EUW/eumaster.csv',low_memory=False)
print(df3.shape)

(2488, 1552)


In [50]:
df = pd.concat([df1,df2,df3])
print(df.shape)

(16615, 1552)


In [51]:
df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participants_6_timeline_damageTakenDiffPerMinDeltas_30-end,participants_7_timeline_csDiffPerMinDeltas_30-end,participants_7_timeline_csDiffPerMinDeltas_20-30,participants_7_timeline_xpDiffPerMinDeltas_30-end,participants_7_timeline_xpDiffPerMinDeltas_20-30,participants_7_timeline_damageTakenDiffPerMinDeltas_30-end,participants_7_timeline_damageTakenDiffPerMinDeltas_20-30,participants_8_timeline_csDiffPerMinDeltas_30-end,participants_8_timeline_xpDiffPerMinDeltas_30-end,participants_8_timeline_damageTakenDiffPerMinDeltas_30-end
0,4681072641,EUW1,1593268978835,1299,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4707844099,EUW1,1594758284675,1438,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4679991311,EUW1,1593201304458,1826,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4648730648,EUW1,1591566971144,1523,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4632641560,EUW1,1590840449379,1908,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,4656414258,EUW1,1591957338230,1417,420,11,13,10.12.324.5925,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2484,4682563131,EUW1,1593355545520,1479,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2485,4684594750,EUW1,1593456408458,1970,440,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,-0.45,NaN,-372.75,NaN,266.95,NaN,NaN,NaN
2486,4675157584,EUW1,1592935405722,1610,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Drop columns of single distinct value or >25% null

In [52]:
# keep only classic games

df = df[df['gameMode'] == "CLASSIC"]

In [53]:
droplist = []

# drop if one distinct value
for col in df.columns:
    if len(df[col].unique())==1:
        droplist.append(col)

# drop if null values are more than 25% of column        
for col in df.columns:
    if (np.mean(df[col]!=df[col]) > .25):
        droplist.append(col)
        
df = df[[col for col in df.columns if col not in droplist]]

In [56]:
df

,gameId,gameCreation,gameDuration,queueId,gameVersion,teams_0_win,teams_0_firstBlood,teams_0_firstTower,teams_0_firstInhibitor,teams_0_firstBaron,...,participants_5_stats_firstInhibitorKill,participants_5_stats_firstInhibitorAssist,participants_6_stats_firstInhibitorKill,participants_6_stats_firstInhibitorAssist,participants_7_stats_firstInhibitorKill,participants_7_stats_firstInhibitorAssist,participants_8_stats_firstInhibitorKill,participants_8_stats_firstInhibitorAssist,participants_9_stats_firstInhibitorKill,participants_9_stats_firstInhibitorAssist
0,4681072641,1593268978835,1299,420,10.13.326.4870,Fail,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4707844099,1594758284675,1438,420,10.14.327.8505,Fail,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
2,4679991311,1593201304458,1826,420,10.13.326.4870,Win,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,4648730648,1591566971144,1523,420,10.11.322.2991,Fail,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,4632641560,1590840449379,1908,420,10.11.322.2991,Win,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,4656414258,1591957338230,1417,420,10.12.324.5925,Fail,False,False,False,False,...,False,True,True,False,False,True,False,False,False,False
2484,4682563131,1593355545520,1479,420,10.13.326.4870,Win,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
2485,4684594750,1593456408458,1970,440,10.13.326.4870,Win,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
2486,4675157584,1592935405722,1610,420,10.12.325.6222,Win,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [57]:
for col in df.columns:
    print(col + ": " + str(df[col].dtype))

gameId: int64
gameCreation: int64
gameDuration: int64
queueId: int64
gameVersion: object
teams_0_win: object
teams_0_firstBlood: bool
teams_0_firstTower: bool
teams_0_firstInhibitor: bool
teams_0_firstBaron: bool
teams_0_firstDragon: bool
teams_0_firstRiftHerald: bool
teams_0_towerKills: int64
teams_0_inhibitorKills: int64
teams_0_baronKills: int64
teams_0_dragonKills: int64
teams_0_riftHeraldKills: int64
teams_0_bans_0_championId: float64
teams_0_bans_0_pickTurn: float64
teams_0_bans_1_championId: float64
teams_0_bans_1_pickTurn: float64
teams_0_bans_2_championId: float64
teams_0_bans_2_pickTurn: float64
teams_0_bans_3_championId: float64
teams_0_bans_3_pickTurn: float64
teams_0_bans_4_championId: float64
teams_0_bans_4_pickTurn: float64
teams_1_win: object
teams_1_firstBlood: bool
teams_1_firstTower: bool
teams_1_firstInhibitor: bool
teams_1_firstBaron: bool
teams_1_firstDragon: bool
teams_1_firstRiftHerald: bool
teams_1_towerKills: int64
teams_1_inhibitorKills: int64
teams_1_baronKi

In [58]:
# fill after to allow int64 reconversion
df = df.fillna(0)

In [59]:
# drop if null values are more than 25% of column

for col in df.columns:
    if (df[col].dtypes == np.float64) and (sum(df[col]%1) == 0):
        df[col] = df[col].astype(np.int64)
    if (df[col].dtypes == bool):
        df[col] = df[col].astype(int)

In [60]:
df[['teams_0_win','teams_1_win']] = df[['teams_0_win','teams_1_win']].replace(['Win','Fail'],[1,0])

In [61]:
df

,gameId,gameCreation,gameDuration,queueId,gameVersion,teams_0_win,teams_0_firstBlood,teams_0_firstTower,teams_0_firstInhibitor,teams_0_firstBaron,...,participants_5_stats_firstInhibitorKill,participants_5_stats_firstInhibitorAssist,participants_6_stats_firstInhibitorKill,participants_6_stats_firstInhibitorAssist,participants_7_stats_firstInhibitorKill,participants_7_stats_firstInhibitorAssist,participants_8_stats_firstInhibitorKill,participants_8_stats_firstInhibitorAssist,participants_9_stats_firstInhibitorKill,participants_9_stats_firstInhibitorAssist
0,4681072641,1593268978835,1299,420,10.13.326.4870,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4707844099,1594758284675,1438,420,10.14.327.8505,0,0,0,0,0,...,False,False,False,False,False,True,False,True,False,False
2,4679991311,1593201304458,1826,420,10.13.326.4870,1,1,1,1,1,...,False,False,False,False,False,False,False,False,False,False
3,4648730648,1591566971144,1523,420,10.11.322.2991,0,1,0,0,0,...,False,True,False,False,False,False,False,False,False,False
4,4632641560,1590840449379,1908,420,10.11.322.2991,1,1,0,1,1,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,4656414258,1591957338230,1417,420,10.12.324.5925,0,0,0,0,0,...,False,True,True,False,False,True,False,False,False,False
2484,4682563131,1593355545520,1479,420,10.13.326.4870,1,1,1,1,0,...,False,False,False,False,False,False,False,False,False,False
2485,4684594750,1593456408458,1970,440,10.13.326.4870,1,0,0,1,1,...,False,False,False,False,False,False,False,False,False,False
2486,4675157584,1592935405722,1610,420,10.12.325.6222,1,1,1,1,1,...,False,False,False,False,False,False,False,False,False,False


In [62]:
df.to_csv('cleanEU.zip', index=False, compression = dict(method='zip',archive_name='cleanEU.csv'))